@misc{lee2024nvembed,
      title={NV-Embed: Improved Techniques for Training LLMs as Generalist Embedding Models}, 
      author={Chankyu Lee and Rajarshi Roy and Mengyao Xu and Jonathan Raiman and Mohammad Shoeybi and Bryan Catanzaro and Wei Ping},
      year={2024},
      eprint={2405.17428},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}


https://huggingface.co/nvidia/NV-Embed-v1

In [1]:
import os
os.chdir('../')
os.getcwd()

'/Users/dennis/repos/study/mt/mt_rdf'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/raw/ppaths_join_nl.csv')

In [4]:
access_token=os.environ.get('HUGGINGFACE_TOKEN')

In [16]:
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import login
from tqdm import tqdm

In [6]:
login(token=access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/dennis/.cache/huggingface/token
Login successful


In [7]:
# load model with tokenizer
model = AutoModel.from_pretrained('nvidia/NV-Embed-v1', trust_remote_code=True) #, use_auth_token=True, token=access_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
task_name_to_instruct = {"example": "Given a Natural Language Question based on a SPARQL Query, return '1' if the original SPARQL Query has a faster runtime if you turn off the virtuoso optmizer or '0' otherwise.",}

In [9]:
df

,query_id,query,label
0,P180,Which municipal arrondissements are located in...,0
1,P181,What is the field of occupation of Stéphane Du...,1
2,P182,Which administrative territorial entities are ...,0
3,P183,What is the ISO 3166-2 code of the administrat...,0
4,P184,What is the ISO 3166-2 code of the administrat...,0
...,...,...,...
209,P399,Which paintings created by a member of the Roy...,0
210,P400,What are the locations and types of mountains ...,0
211,P401,What are the different types of musical instru...,0
212,P402,In which administrative territorial entities d...,0


In [10]:
df.iloc[1]['query']

'What is the field of occupation of Stéphane Dujarric?'

In [11]:
df['query'][:10]

0    Which municipal arrondissements are located in...
1    What is the field of occupation of Stéphane Du...
2    Which administrative territorial entities are ...
3    What is the ISO 3166-2 code of the administrat...
4    What is the ISO 3166-2 code of the administrat...
5       Which people were born in a geographic region?
6    What is the administrative territorial entity ...
7    What are all the instances of automobile model...
8    Which administrative territorial entity does t...
9    Which lighthouses are located in administrativ...
Name: query, dtype: object

In [13]:
query_prefix = "Instruct: "+task_name_to_instruct["example"]+"\nQuery: "

queries = list(df['query'])

In [12]:
max_length = 250

In [17]:
def process_in_batches(entries, batch_size):
    encoded_queries = []
    for i in tqdm(range(0, len(entries), batch_size)):
        batch = entries[i:i + batch_size]
        encoded_batch = model.encode(batch, instruction=query_prefix, max_length=max_length)
        encoded_queries.extend(encoded_batch)
    return encoded_queries


In [18]:
batch_size = 10

encoded_qs = process_in_batches(queries, batch_size)

  0%|          | 0/22 [00:00<?, ?it/s]/Users/dennis/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v1/7f6188488250b5bd3a334d93dfce0f1291f240e3/modeling_nvembed.py:345: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/Users/dennis/repos/study/mt/mt_rdf/env/lib/python3.9/site-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
100%|██████████| 22/22 [54:15<00:00, 148.00s/it]


In [20]:
torch.save(encoded_qs, 'data/processed/ppaths_join/ppaths_join_nl_encoded.pt')

In [17]:
len(query_embeddings[0])

4096

In [23]:
double_df = df.copy()
double_df = pd.concat([double_df] *2, ignore_index=True)

In [27]:
double_df['label'].sum()

50

In [26]:
df['label'].sum()

25